# *Required Libraries*

In [1]:
import spacy
import json
from spacy.util import filter_spans
from spacy.tokens import DocBin
from tqdm import tqdm

# *Loading Data*

In [2]:
with open('Data/train/train_data.json','rb') as f:
    train_data=json.load(f)

# *Building the Model*

In [6]:
nlp = spacy.blank('en')
doc_bin = DocBin()
count=0
for training_example in tqdm(train_data):
    text = training_example['text']
    entities = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in entities:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)
doc_bin.to_disk("train.spacy")

100%|██████████| 220/220 [00:01<00:00, 131.72it/s]


In [8]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [2]:
! python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

^C


✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     86.50    0.00    0.00    0.00    0.00
  0     200          0.33    474.16    0.00    0.00    0.00    0.00
⚠ Aborting and saving the final best model. Encountered exception:
FileNotFoundError(2, 'The system cannot find the path specified')


Traceback (most recent call last):
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 135, in train
    raise e
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 123, in train
    save_checkpoint(is_best_checkpoint)
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\training\loop.py", line 79, in save_checkpoint
    before_to_disk(nlp).to_disk(output_path / DIR_MODEL_LAST)
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\language.py", line 2120, in to_disk
    util.to_disk(path, serializers, exclude)
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\util.py", line 1373, in to_disk
    path.mkdir()
  File "c:\Users\Mitesh Gupta\AppData\Local\Programs\Python\Python311\Lib\pathlib.py", line 1116, in mkdir
    os.mkdir(self, mo